# Bengali AI - Image Processing

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob2
import gc
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter
from itertools import chain
from functools import reduce
# from src.jupyter import grid_df_display, combination_matrix

pd.set_option('display.max_columns',   500)
pd.set_option('display.max_colwidth',   -1)

%load_ext autoreload
%autoreload 2

In [ ]:
### Only process a single parquet file for the sake of Kaggle Kernel Memory Limits
train = None; gc.collect()
train = pd.concat([ pd.read_parquet(filename) for filename in sorted(glob2.glob('../input/bengaliai-cv19/train_image_data_0.parquet')) ])
# train = pd.concat([ pd.read_parquet(filename) for filename in sorted(glob2.glob('../input/bengaliai-cv19/train_*.parquet')) ])
# test  = pd.concat([ pd.read_parquet(filename) for filename in sorted(glob2.glob('../input/bengaliai-cv19/test_*.parquet'))  ])
# train = train[0:8]
train_drop = train.copy()
train_drop.set_index('image_id', inplace=True, drop=True)
gc.collect()
train.head()

In [ ]:
img = train.iloc[0,1:].values.astype('uint8').reshape(137,236); (img.shape, img.dtype)
display( plt.imshow(img, cmap='gray') )

In [ ]:
import cv2
from tqdm import tqdm


# Source: https://www.kaggle.com/c/bengaliai-cv19/discussion/122731
# Source: https://www.kaggle.com/maxlenormand/cropping-to-character-resizing-images
def crop_and_resize_images(df, resized_df, resize_size=236):
    cropped_imgs = {}
    for img_id in tqdm(range(df.shape[0])):
        img = resized_df[img_id]
        _, thresh = cv2.threshold(img, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]
        
        idx = 0 
        ls_xmin = []
        ls_ymin = []
        ls_xmax = []
        ls_ymax = []
        for cnt in contours:
            idx += 1
            x,y,w,h = cv2.boundingRect(cnt)
            ls_xmin.append(x)
            ls_ymin.append(y)
            ls_xmax.append(x + w)
            ls_ymax.append(y + h)
        xmin = min(ls_xmin)
        ymin = min(ls_ymin)
        xmax = max(ls_xmax)
        ymax = max(ls_ymax)

        roi = img[ymin:ymax,xmin:xmax]
        resized_roi = cv2.resize(roi, (resize_size, resize_size))
        cropped_imgs[df.image_id[img_id]] = resized_roi.reshape(-1)
        
    resized = pd.DataFrame(cropped_imgs).T.reset_index()
    resized.columns = resized.columns.astype(str)
    resized.rename(columns={'index':'image_id'},inplace=True)
    return resized #out_df

In [ ]:
df_0       = pd.read_parquet('../input/bengaliai-cv19/train_image_data_0.parquet')
resized    = df_0.iloc[:, 1:].values.astype(np.uint8).reshape(-1, 137,236)
cropped_df = crop_and_resize_images(df_0, resized, 100)
train_images  = cropped_df.iloc[0,1:].values.astype(np.uint8).reshape(-1,100,100,1)
# train_images  = train.iloc[0,1:].values.astype(np.uint8).reshape(-1,137,236)
# train_cropped = crop_and_resize_images(train, train_images, 100)
# train_cropped
cropped_df

In [ ]:



# DOCS: https://keras.io/preprocessing/image/#imagedatagenerator-class
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # featurewise_center=False,             # No visible effect in plt.imgshow()
    # samplewise_center=False,              # No visible effect in plt.imgshow()
    # featurewise_std_normalization=False,  # No visible effect in plt.imgshow()
    # samplewise_std_normalization=False,   # No visible effect in plt.imgshow() 
    # zca_whitening=True,                   # Kaggle, insufficent memory

    rotation_range=45/2,
#     width_shift_range=0.2,
#     height_shift_range=0.2,        
    brightness_range=(0.5,2),
    shear_range=45/2,
#     zoom_range=[-0.2, 0],
    fill_mode='constant',
    cval=255,        
)
train_datagen.fit(
    train_images    
)
train_generator = train_datagen.flow(
    train_images,
    subset="training",
    shuffle=False,  
    batch_size=8,        
)
gc.collect()

index = 0
fig, ax = plt.subplots(figsize=(20, 10))    
plt.axis('off')
for batch in train_generator:
    for img in batch:
        index += 1
        plt.subplot(8, 8, index)
        plt.imshow(img.reshape(100,100), cmap='gray')        
        # plt.imshow(img.reshape(137,236), cmap='gray')
    if index >= 8*8: break